In [32]:
import requests
import passwords as pw
import pandas as pd


In [33]:
def get_puuid(game_name, tag_line, api_key):
    api_url = (f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key={api_key}")
    resp = requests.get(api_url)
    player_info = resp.json()
    return player_info['puuid']


In [34]:
def get_active_game(puuid, api_key):
    headers = {"X-Riot-Token": api_key}
    api_url = (f"https://na1.api.riotgames.com/lol/spectator/v5/active-games/by-summoner/{puuid}")
    resp = requests.get(api_url, headers = headers)
    game_data = resp.json()

    champion_ids = [player['championId'] for player in game_data['participants']]
    return champion_ids


In [35]:
def champion_names(champ_ids):
    version = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
    api_url = f"https://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/champion.json"
    champs = requests.get(api_url).json()["data"]
    id_to_name = {int(name["key"]): champ for champ, name in champs.items()}
    
    champ_names = [id_to_name.get(id, "Unknown") for id in champ_ids]
    print("Champions in game:", champ_names)

In [36]:
def get_match_ids(puuid, api_key):
    api_url = ("https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/" +
        puuid + "/ids?start=0&count=10&api_key=" + api_key
    )
    resp = requests.get(api_url)
    match_ids = resp.json()
    return match_ids

In [37]:
def get_match_data(match_id, api_key):
    api_url = ("https://americas.api.riotgames.com/lol/match/v5/matches/" + match_id +
               "?api_key=" + api_key)
    try:
        resp = requests.get(api_url)
        match_data = resp.json()
        return match_data
    except:
        return None

In [38]:
puuid = get_puuid("McDonaldsManager", "mid", pw.API_KEY)
puuid

'bCnPTf3VMxhRVLuPR3147HhV_hUrIRYSFt4fWgFUcIivP9tFWcJfuRWLwIliB8qChi9cIvcLztpaFA'

In [54]:
dataframe = []
match_ids = get_match_ids(pw.dl_PUUID, pw.API_KEY)
for match in match_ids:
    match_data = get_match_data(match, pw.API_KEY)
    if not match_data:
        continue

    participants = match_data['info']['participants']
    champions = [player['championName'] for player in participants]
    
    stat = {'match': match}
    for i, champ in enumerate(champions):
        stat[i] = champ
    
    dataframe.append(stat)
    


In [55]:
df = pd.DataFrame(dataframe)
df

,match,0,1,2,3,4,5,6,7,8,9
0,NA1_5311257006,Yone,Talon,Syndra,Jhin,TwistedFate,Riven,LeeSin,Sylas,Seraphine,Janna
1,NA1_5311227228,Gwen,LeeSin,Akali,MissFortune,Nautilus,Sett,Rengar,TwistedFate,Tristana,Yuumi
2,NA1_5311203254,Volibear,Udyr,TwistedFate,Ezreal,Soraka,Gwen,Zed,Yone,Lux,Ashe
3,NA1_5311170342,Kindred,Zac,Galio,Varus,Elise,Ambessa,Viego,TwistedFate,Ekko,Nautilus
4,NA1_5311141585,Malphite,MasterYi,TwistedFate,Tristana,Leona,Camille,Viego,Zed,Malzahar,Renata
5,NA1_5311116625,Renekton,Shaco,Taliyah,Jhin,Thresh,Camille,Hecarim,TwistedFate,Ezreal,Nami
6,NA1_5311093043,Sylas,LeeSin,Kayle,Lucian,JarvanIV,Vladimir,Hecarim,TwistedFate,Senna,Alistar
7,NA1_5311069610,Irelia,Yorick,TwistedFate,Vayne,Milio,Chogath,Ekko,Yasuo,MissFortune,Nidalee
8,NA1_5311049753,DrMundo,Elise,Nunu,Jhin,Pyke,Renekton,Sejuani,Vladimir,Akshan,Morgana
9,NA1_5311023309,Singed,Nocturne,Viktor,Jhin,Brand,Renekton,DrMundo,Xerath,Sivir,Alistar
